In [ ]:
import pandas as pd

# Create a simple DataFrame with numbers
df = pd.DataFrame({'value': [1, 2, 3, 4, 5]})

# Save to CSV locally
df.to_csv('numbers.csv', index=False)

print("✅ File 'numbers.csv' created in local notebook directory.")


In [ ]:
import boto3

# Replace these with your bucket name and desired S3 key (path)
bucket_name = "mlops-sandbox-bucket"
s3_key = "test-data/numbers.csv"  # S3 path where the file will be uploaded

# Local file path in your notebook environment
local_file_path = "numbers.csv"

# Create an S3 client
s3 = boto3.client("s3")

# Upload the file
s3.upload_file(local_file_path, bucket_name, s3_key)

print(f"Uploaded {local_file_path} to s3://{bucket_name}/{s3_key}")


In [ ]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from my_inference_spec import MyInferenceSpec
from sagemaker import get_execution_role

role = get_execution_role()

# Schema definition (auto-handles content-type)
sample_input = "1,2,3,4,5"
sample_output = {"sum": 15}

schema = SchemaBuilder(sample_input=sample_input, sample_output=sample_output)

builder = ModelBuilder(
    role_arn=role,
    model_path=".",  # Just needs to be any path, not actually used for model files
    inference_spec=MyInferenceSpec(),  # Here's your class
    schema_builder=schema,
    mode="SAGEMAKER_ENDPOINT",
    instance_type="ml.t3.medium"
)

# Deploy the model
model = builder.build()
predictor = model.deploy()


In [ ]:
s3://mlops-sandbox-bucket/med_xgb_data.parquet

In [ ]:
import pandas as pd
import boto3
import os

# S3 configuration
bucket_name = "mlops-sandbox-bucket"
s3_key = "med_xgb_data.parquet"
local_file = "data_sample.parquet"  # save locally

# Download from S3 using boto3
s3 = boto3.client('s3')
s3.download_file(bucket_name, s3_key, local_file)

# Read only first 100 rows
df = pd.read_parquet(local_file)
df_sample = df.head(100)

print(df_sample.head())


In [ ]:
import pandas as pd
import xgboost as xgb
import boto3
import joblib
import os

# === 1. Load your data ===
df = pd.read_parquet("data_sample.parquet")  # or use your existing DataFrame directly
X = df.drop("target", axis=1)
y = df["target"]

# === 2. Train an XGBoost model ===
model = xgb.XGBClassifier()
model.fit(X, y)

# === 3. Save the model locally ===
model_dir = "xgb_model"
os.makedirs(model_dir, exist_ok=True)
model_file = os.path.join(model_dir, "model.pkl")
joblib.dump(model, model_file)

print(f"Model saved locally at {model_file}")

# === 4. Upload model to S3 ===



In [ ]:
bucket = "mlops-sandbox-bucket"
s3_key = "xgb-model/model.pkl"

s3 = boto3.client("s3")
s3.upload_file(model_file, bucket, s3_key)

print(f"Model uploaded to s3://{bucket}/{s3_key}")

In [ ]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.serve.spec.inference_spec import InferenceSpec
# from sagemaker.serve.builder.mode import mode
from sagemaker import get_execution_role

role = get_execution_role()

builder = ModelBuilder(
    mode="SAGEMAKER_ENDPOINT",
    model_path="s3://mlops-sandbox-bucket/xgb-endpoint/model.pkl",  # where model.pkl is
    inference_spec=InferenceSpec(predict_fn="inference:predict_fn"),
    schema_builder=SchemaBuilder(
        input=X.iloc[:2].to_dict(orient="records"),  # small sample
        output={"prediction": [0, 1]}  # expected shape
    ),
    role_arn=role,
    dependencies={"auto": True}  # includes inference.py automatically
)

# Deploy endpoint
predictor = builder.deploy()

# Test prediction
result = predictor.predict(X.iloc[:2].to_dict(orient="records"))
print(result)


In [2]:
import pandas as pd
import xgboost as xgb
import boto3
import joblib
import os

# === 1. Load your data ===
df = pd.read_parquet("data_sample.parquet")  # or use your existing DataFrame directly
X = df.drop("target", axis=1)
y = df["target"]


In [8]:
y.iloc[:1][0]
X.iloc[:1]

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99
0,0.584852,0.755651,0.368659,0.119589,0.580436,0.527097,0.653506,0.607716,0.609799,0.653168,...,0.088202,0.24947,0.009453,0.83262,0.264304,0.460975,0.390494,0.830921,0.601923,0.156969


In [ ]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

In [9]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker import get_execution_role

role = get_execution_role()

model_builder = ModelBuilder(
    model="s3://mlops-sandbox-bucket/xgb-endpoint/model.pkl",
    schema_builder=SchemaBuilder(X.iloc[:1], y.iloc[:1][0]),
    role_arn="execution-role",
)


In [10]:
model = model_builder.build()

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.


[06/13/25 05:37:05] INFO     Either inference spec or model is provided. ModelBuilder is not   ]8;id=996384;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py\model_builder.py]8;;\:]8;id=686525;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py#845\845]8;;\
                             handling MLflow model input                                                           

INFO:sagemaker_core.main.utils:Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model = model_builder.build()                                                                │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py:1022 in build   │
│                                                                                                  │
│   1019 │   │   │   self.inference_spec, (CustomOrchestrator, AsyncCustomOrchestrator)            │
│   1020 │   │   ):                                                                                │
│   1021 │   │   │   self.serve_settings = self._get_serve_setting()                               │
│ ❱ 1022 │   │   │   return self._build_single_modelbuilder(                                       │
│   1023 │   │   │   │   mode=mode,                                                                │
│   1024 │   │   │   │   role_arn=self.role_arn,                                                   │
│   1025 │   │   │   │   sagemaker_session=sagemaker_session,                                      │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/utils/telemetry_logger.py:117 in wrapper │
│                                                                                                  │
│   114 │   │   │   │   failure_reason = str(e)                                                    │
│   115 │   │   │   │   failure_type = e.__class__.__name__                                        │
│   116 │   │   │   except Exception as e:  # pylint: disable=W0703                                │
│ ❱ 117 │   │   │   │   raise e                                                                    │
│   118 │   │   │                                                                                  │
│   119 │   │   │   stop_timer = perf_counter()                                                    │
│   120 │   │   │   elapsed = stop_timer - start_timer                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/utils/telemetry_logger.py:105 in wrapper │
│                                                                                                  │
│   102 │   │   │                                                                                  │
│   103 │   │   │   start_timer = perf_counter()                                                   │
│   104 │   │   │   try:                                                                           │
│ ❱ 105 │   │   │   │   response = func(self, *args, **kwargs)                                     │
│   106 │   │   │   except (                                                                       │
│   107 │   │   │   │   ModelBuilderException,                                                     │
│   108 │   │   │   │   exceptions.CapacityError,                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py:1314 in         │
│ _build_single_modelbuilder                                                                       │
│                                                                                                  │
│   1311 │   │   │   if self._is_djl():                      

In [1]:
import joblib

# Load the model from the file
model = joblib.load("xgb_model/model.pkl")

In [4]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker import get_execution_role

role = get_execution_role()
model_builder_local = ModelBuilder(
    model=model,
    schema_builder=SchemaBuilder(X, y),
    role_arn=execution-role,
    mode=Mode.LOCAL_CONTAINER
)
xgb_local_builder = model_builder_local.build()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 role = get_execution_role()                                                                 │
│    6 model_builder_local = ModelBuilder(                                                         │
│    7 │   model=model,                                                                            │
│ ❱  8 │   schema_builder=SchemaBuilder(X, y),                                                     │
│    9 │   role_arn=execution-role,                                                                │
│   10 │   mode=Mode.LOCAL_CONTAINER                                                               │
│   11 )                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/schema_builder.py:147 in         │
│ __init__                                                                                         │
│                                                                                                  │
│   144 │   │   │   )                                                                              │
│   145 │   │   │   self.custom_output_translator = output_translator                              │
│   146 │   │   else:                                                                              │
│ ❱ 147 │   │   │   self._output_deserializer = self._get_deserializer(sample_output)              │
│   148 │   │   │   self.output_serializer = self._get_inverse(self._output_deserializer)          │
│   149 │   │   │                                                                                  │
│   150 │   │   │   self.output_deserializer = DeserializerWrapper(                                │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/schema_builder.py:200 in         │
│ _get_deserializer                                                                                │
│                                                                                                  │
│   197 │   │   if _is_jsonable(obj):                                                              │
│   198 │   │   │   return JSONDeserializer()                                                      │
│   199 │   │                                                                                      │
│ ❱ 200 │   │   raise ValueError(                                                                  │
│   201 │   │   │   (                                                                              │
│   202 │   │   │   │   "SchemaBuilder cannot determine deserializer of type %s "                  │
│   203 │   │   │   │   "Please provide your own marshalling code"                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: SchemaBuilder cannot determine deserializer of type <class 'pandas.core.series.Series'> Please provide 
your own marshalling codeto SchemaBuilder via CustomPayloadTranslator

In [6]:
from sagemaker.serve import CustomPayloadTranslator

# request translator
class MyRequestTranslator(CustomPayloadTranslator):
    # This function converts the payload to bytes - happens on client side
    def serialize_payload_to_bytes(self, payload: object) -> bytes:
        # converts the input payload to bytes

        return  //return object as bytes

    # This function converts the bytes to payload - happens on server side
    def deserialize_payload_from_stream(self, stream) -> object:
        # convert bytes to in-memory object

        return //return in-memory object

# response translator
class MyResponseTranslator(CustomPayloadTranslator):
    # This function converts the payload to bytes - happens on server side
    def serialize_payload_to_bytes(self, payload: object) -> bytes:
        # converts the response payload to bytes
        return //return object as bytes

    # This function converts the bytes to payload - happens on client side
    def deserialize_payload_from_stream(self, stream) -> object:

        return //return in-memory object

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│         return  //return object as bytes                                                         │
│                 ▲                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax